In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 35.5 MB/s eta 0:00:00


In [2]:
import mysql.connector
import pandas as pd

In [3]:
conn = mysql.connector.connect(
    host="18.136.157.135",
    user="dm_team17",
    password="9o!etlre=itA",
    database="project_shopping_trends"
)

df = pd.read_sql("SELECT * FROM shopping_table", conn)
df.head()

/tmp/ipython-input-1514513458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM shopping_table", conn)


,Customer_ID,Age,Gender,Item_Purchased,Category,Purchase_Amount_USD,Location,Size,Color,Season,Review_Rating,Subscription_Status,Payment_Method,Shipping_Type,Discount_Applied,Promo_Code_Used,Previous_Purchases,Preferred_Payment_Method,Frequency_of_Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item_Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase_Amount_USD       3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review_Rating             3900 non-null   float64
 11  Subscription_Status       3900 non-null   object 
 12  Payment_Method            3900 non-null   object 
 13  Shipping_Type             3900 non-null   object 
 14  Discount

In [5]:
discount_category = df.groupby(
    ['Category', 'Discount_Applied']
)['Purchase_Amount_USD'].mean()

discount_category

Category     Discount_Applied
Accessories  No                  60.889527
             Yes                 58.489871
Clothing     No                  60.223658
             Yes                 59.752394
Footwear     No                  59.079412
             Yes                 61.799228
Outerwear    No                  58.655556
             Yes                 55.319444
Name: Purchase_Amount_USD, dtype: float64

In [ ]:
age_spending = df.groupby('Age')['Purchase_Amount_USD'].mean()
age_spending.head()

,Purchase_Amount_USD
Age,
18,59.985507
19,61.000000
20,55.225806
21,64.101449
22,60.772727


In [ ]:
season_spending = df.groupby('Season')['Purchase_Amount_USD'].sum()
season_spending

,Purchase_Amount_USD
Season,
Fall,60018
Spring,58679
Summer,55777
Winter,58607


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Discount_Applied'] = le.fit_transform(df['Discount_Applied'])

X = df[['Age', 'Purchase_Amount_USD', 'Previous_Purchases']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

df[['Age','Purchase_Amount_USD','Cluster']].head()

,Age,Purchase_Amount_USD,Cluster
0,55,53,0
1,19,64,1
2,50,73,0
3,21,90,2
4,45,49,0


In [ ]:
cluster_summary = df.groupby('Cluster')[
    ['Age', 'Purchase_Amount_USD', 'Previous_Purchases', 'Review_Rating']
].mean()

cluster_summary

,Age,Purchase_Amount_USD,Previous_Purchases,Review_Rating
Cluster,,,,
0,59.698917,58.569012,28.451962,3.735792
1,36.332817,59.765480,11.018576,3.748994
2,32.469027,61.326549,37.684071,3.769558
